# 3. 함수 정의와 호출
3장에서는 함수 선언과 호출이 코틀린에선 어떠한지 살펴본다. 이 과정에서 확장함수와 프로퍼티를 사용해 코틀린스럽게 코드를 작성하는 법도 알아본다.

## 코틀린에서 컬렉션 만들기
리스트와 맵을 만들어보자.

In [2]:
fun main() {
    val list = listOf(1, 2, 3)
    val map = mapOf(1 to "one",
        2 to "two",
        3 to "three")

    println(list.javaClass) // javaClass는 자바에서 getClass()에 해당하는 코틀린 표현임.
    println(map.javaClass) //LinkedHashMap
}

main()

class java.util.Arrays$ArrayList
class java.util.LinkedHashMap


> 참고로 여기서 `to` 는 걍 함수다. (키워드가 아니다!) `Pair` 객체를 생성하는 infix함수임.

> 코틀린 '표현'이 정확히 무엇을 말하는가? 일단 `javaClass` 는 프로퍼티가 아니다. 이건 말 그대로 '표현'이고, 코틀린 컴파일러가 내부적으로 `this::class.java` 로 치환한다.
> 즉 코틀린 컴파일러가 이해할 수 있는 문법적 요소임. 표준 함수도, 확장 프로퍼티도 아니다. 컴파일러가 AST단계에서 치환함.

위에서 `println()` 을 찍어보면 알 수 있듯이 **코틀린의 list와 map은 자바의 컬렉션을 사용하고 있다.**
하지만 자바와의 차이점으로 코틀린의 컬렉션 인터페이스는 **기본값이 읽기 전용이라는 점임.** (함수형 프로그래밍을 고려한건가?)
이런 읽기 전용 컬렉션과 이에 일대일로 대응하는 가변 인터페이스들이 따로 있다.

코틀린 컬렉션은 기본적으로 자바의 컬렉션을 그대로 쓰고 있긴 하지만, 몇가지 기능이 더 추가된 인터페이스를 갖고 있.

In [3]:
fun main() {
    val Strings = listOf("first", "second", "fourteenth")
    println(Strings.last()) //리스트의 마지막 원소를 가져온다.

    println(Strings.shuffled()) //원소를 뒤섞은 버전의 리스트를 반환한다!

    val numbers = setOf(1, 14, 2)
    println(numbers.max())
    println(numbers.sum()) //수로 이뤄진 컬렉션일 경우 합계도 구해준다!
}

main()

fourteenth
[first, fourteenth, second]
14
17


## 함수를 호출하기 쉽게 만들기
자바 컬렉션에는 기본적으로 `toString()` 구현이 포함되어 있다. 하지만 대부분 이걸 직접 오버라이딩해서 사용하거나 구아바, 아파치 커먼즈 같은 라이브러리를 사용한다.

우선 코틀린의 이점이 없는 방식으로 직접 구현해보자.

In [4]:
fun <T> joinToString(
    collection: Collection<T>,
    separator: String,
    prefix: String,
    postfix: String
): String {

    val result = StringBuilder(prefix)

    for((index, element) in collection.withIndex()) {
        if (index > 0) result.append(separator)
        result.append(element)
    }

    result.append(postfix)
    return result.toString()
}

fun main() {
    val list = listOf(1, 2, 3)
    println(joinToString(list, "; ", "(", ")"))
}

main() //(1; 2; 3)

(1; 2; 3)


위의 코드가 잘 동작하긴 하지만 함수를 사용하는 쪽을 보면 아규먼트가 네개나 되어서 복잡하다. 이걸 개선하려면 함수 시그니처를 어떻게 만들어야 할까?

### 이름붙은 인자
위처럼 타입이 전부 String인데다가, 파라미터가 4개나 되면 각 파라미터가 어떤 역할을 하는지 함수 시그니처를 직접 찾아가기 전까진 헤매기 쉽다.
그래서 코틀린은 다른 언어가 그렇듯이 이름이 있는 아규먼트를 지원한다.